In [29]:
from dotenv import load_dotenv
load_dotenv()

from typing import TypedDict, Literal, Annotated
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages



In [30]:
llm = init_chat_model("google_genai:gemini-2.0-flash")


class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [31]:
message = {"role": "user", "content": "Who invented the light bulb? Print only the name of the inventor."}
  
responce = graph.invoke({"messages": [message]})

responce["messages"]

[HumanMessage(content='Who invented the light bulb? Print only the name of the inventor.', additional_kwargs={}, response_metadata={}, id='4a4f5ea5-118e-4a81-b667-3be6a01ada86'),
 AIMessage(content='Thomas Edison', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--adb6cda4-083c-4e97-99b1-238683b23436-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [32]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# UI elements
input_box = widgets.Text(placeholder="Type a message...", description="You:", continuous_update=False)
submit_button = widgets.Button(description="Send")
output_area = widgets.Output()

ui = widgets.VBox([input_box, submit_button, output_area])



In [33]:
# Initial state
state = {"messages": []}

def on_submit(_):
    global state
    user_input = input_box.value.strip()
    input_box.value = ""

    if user_input.lower() in ["exit", "quit"]:
        with output_area:
            print("Session ended.")
        return

    # Append user message
    state["messages"].append({"role": "user", "content": user_input})

    # Run chat graph
    state = graph.invoke(state)

    # Extract last message (whether raw dict or AIMessage object)
    last_msg = state["messages"][-1]
    if isinstance(last_msg, dict):
        bot_reply = last_msg.get("content", "No reply")
    else:
        bot_reply = last_msg.content
    with output_area:
        print(f"You: {user_input}")
        print(f"Bot: {bot_reply}")

# Button click
submit_button.on_click(on_submit)

# ENTER key (no deprecated .on_submit)
def on_text_submit(change):
    if change["name"] == "value" and change["type"] == "change" and input_box.value.strip():
        on_submit(None)

input_box.observe(on_text_submit, names="value")




In [34]:
display(ui)